In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import urllib.request
from bs4 import BeautifulSoup as bs
import re
import json, csv

In [47]:
# 각 페이지에 여러 음식 조리법 리스트가 있기 때문에 url 리스트를 만들고, 이곳에 각 음식 조리법의 url을 추가한다.
def url_func(n, m):
    num_range = range(n, m)
    url = "https://www.10000recipe.com/profile/recipe.html?uid=gdubu33&page="
    url_list = []
    for num in num_range:
        req = urllib.request.Request(url + str(num))
        code = urllib.request.urlopen(req).read()
        soup = bs(code, "html.parser")
        
        try:
            res = soup.find(class_='cont_list st3') # 음식 조리법 url
            for i in res.find_all('a'):
                url_tmp = i.get('href')
                url_list.append(url_tmp)
        except(AttributeError):
            pass
        
    return url_list

In [56]:
url_lists = url_func(1, 160)

In [57]:
# 약 3180개의 음식 조리법
len(url_lists)

3180

In [17]:
"""
for url_str in url_lists[:1]:
    ingre_1 = []
    url = "https://www.10000recipe.com"
    url = url + url_str
    req = urllib.request.Request(url)
    code = urllib.request.urlopen(url).read()
    soup = bs(code, "html.parser")
    
    res = soup.find('div', 'cont_ingre2')
    for i in res.find_all('a'):
        ingredient_name = i.get_text().replace('\n', '').replace(' ', '')
        ingre_1.append(ingredient_name)
print(ingre_1)
"""

['오징어3마리', '돼지고기다짐육100g', '두부1/4모', '표고버섯3개', '홍고추1개', '청양고추1개', '대파10cm', '밀가루약간', '달걀1개', '다진마늘1/2숟가락', '간장1숟가락', '소금약간', '후추약간', '참기름1숟가락', '참깨1숟가락', '가리비8개', '다진청고추약간', '다진홍고추약간', '다진양파약간', '피자치즈100g', '초장2숟가락', '낙지6마리', '참깨1숟가락', '참기름1숟가락', '청주1숟가락', '다진마늘1/4숟가락', '다진생강약간', '데리야끼소스약간']


In [54]:
for url_str in url_lists[:1]:
    ingre_1 = []
    url = "https://www.10000recipe.com"
    url = url + url_str
    req = urllib.request.Request(url)
    code = urllib.request.urlopen(url).read()
    soup = bs(code, "html.parser")
    
    res = soup.find('div', 'view2_summary st3')
    menu_summary = res.get_text().replace('\n', '').strip()
    
print(menu_summary)

                오늘 레시피는 눈 깜짝할 사이에 만들어지니까 주목!'스팀'에어프라이어로 만드는 홈포차 레시피 3종!스팀 기능이 추가된 에어프라이어는 겉바속촉 식감에더욱 건강한 요리를 만들 수 있어요.촉촉한 찜요리엔 스팀 모드,바삭한 튀김요리엔 에어프라이어 모드,겉바속촉 요리엔 스팀&프라이 모드!잘 모르겠다면? 재료별 원버튼만 누르면 끝~이 정도면 '만능'이란 말이 절로 떠오를 만하죠.@@오늘 홈술 홈쿡 준비물은 요거다!                6인분 이상 120분 이내 아무나


In [60]:
name_list = []
summary_list = []
ingre_list1 = []
ingre_list2 = []
recipe_list = []

# 160까지만 크롤링 -> 180까지 하면 오류 발생, 아마도 만개의 레시피 웹사이트 문제인 듯
for url_str in url_lists[2000:3000]:
    url = "https://www.10000recipe.com"
    url = url + url_str
    req = urllib.request.Request(url)
    code = urllib.request.urlopen(req).read()
    soup = bs(code, "html.parser")

    ingre1_ = []
    ingre2_ = []
    recipe_ = []
    
    # 음식 메뉴 이름
    res = soup.find('div', 'view2_summary')
    res = res.find('h3')
    menu_name = res.get_text()
    
    # 음식 메뉴 소개글(요약 버전)
    res = soup.find('div', 'view2_summary st3')
    menu_summary = res.get_text().replace('\n', '').strip()
    
    # 음식 재료들
    res = soup.find('div', 'cont_ingre2')
    try:
        for n in res.find_all('a'):
            ingredient_name = n.get_text().replace('\n', '').replace(' ', '') # 고추장 1스푼
            ingredient_unit = ''.join([i for i in ingredient_name if not i.isdigit()]) # 고추장 1스푼 -> 고추장 스푼
                
            ingre1_.append(ingredient_name)
            ingre2_.append(ingredient_unit) 

    except(AttributeError):
        pass
    
    # 음식 조리법
    res = soup.find('div', 'view_step')
    for n in res.find_all('div', 'view_step_cont'):
        recipe_step_txt = n.get_text().replace('\n', '')
        
        recipe_.append(recipe_step_txt)

    name_list.append(menu_name)
    summary_list.append(menu_summary)
    ingre_list1.append(ingre1_)
    ingre_list2.append(ingre2_)
    recipe_list.append(recipe_)

csv_data = {"name": name_list,
            "summary": summary_list,
            "ingre": ingre_list1,
            "ingre_without_num": ingre_list2,
            "recipe": recipe_list}

df = pd.DataFrame(csv_data)

df.to_csv('./db_recipe3.csv', sep=',', na_rep='NaN', encoding='utf-8-sig') # na_rep: NaN 표시값을 바꿔줌 